In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load the dataset
df = pd.read_csv("C:/Users/perso/OneDrive/Desktop/deep_learning/Dl_paper/HomeC.csv")

C:\Users\perso\AppData\Local\Temp\ipykernel_12800\855866583.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/perso/OneDrive/Desktop/deep_learning/Dl_paper/HomeC.csv")


In [3]:
# Convert 'time' column to datetime format and coerce errors
df['time'] = pd.to_datetime(df['time'], errors='coerce')

In [4]:
# Define critical target columns
critical_columns = ['use[kW]', 'House_overall[kW]', 'Furnace_1[kW]', 'Home_office[kW]',
                    'Fridge[kW]', 'Wine_cellar[kW]', 'Kitchen_14[kW]', 'Microwave[kW]', 'Living_room[kW]']

# Define weather-related input columns
weather_columns = ['temperature', 'humidity', 'visibility', 'apparentTemperature', 
                   'pressure', 'windSpeed', 'windBearing', 'dewPoint']

In [5]:
# Drop rows with missing values in critical target columns and forward-fill/backward-fill weather columns
df_cleaned = df.dropna(subset=critical_columns)
df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')



C:\Users\perso\AppData\Local\Temp\ipykernel_12800\3662371642.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')


## House_overall

In [6]:
# Separate input features and target variables
X = df_cleaned[weather_columns]
y = df_cleaned[critical_columns[1]]

In [7]:
print("Shape of X_sequences:", X.shape)
print("Shape of y_sequences:", y.shape)

Shape of X_sequences: (503910, 8)
Shape of y_sequences: (503910,)


In [8]:
# Scale features and target columns
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = y

In [9]:
print("Shape of X_sequences:", X_scaled.shape)
print("Shape of y_sequences:", y_scaled.shape)

Shape of X_sequences: (503910, 8)
Shape of y_sequences: (503910,)


In [10]:
# sliding window for generating data
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [11]:
# Create sequences
time_steps = 30
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, time_steps=time_steps)


In [12]:
print("Shape of X_sequences:", X_sequences.shape)
print("Shape of X_sequences:", y_sequences.shape)

Shape of X_sequences: (503880, 30, 8)
Shape of X_sequences: (503880,)


In [13]:
# divide the sequences into training and testing
# Split the data into train and test sets
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_sequences, y_sequences, test_size=0.2, shuffle=False)

In [14]:
# Debugging: Ensure X_sequences and y_sequences are arrays with expected shapes
print("Shape of X_sequences:", X_train_seq.shape)
print("Shape of y_sequences:", y_train_seq.shape)
print("Data type of X_sequences:", X_train_seq.dtype)
print("Data type of y_sequences:", y_train_seq.dtype)

# Check for NaN values in the sequences
print("NaN values in X_sequences:", np.isnan(X_train_seq).any())
print("NaN values in y_sequences:", np.isnan(y_train_seq).any())

Shape of X_sequences: (403104, 30, 8)
Shape of y_sequences: (403104,)
Data type of X_sequences: float64
Data type of y_sequences: float64
NaN values in X_sequences: False
NaN values in y_sequences: False


In [15]:
X_train_seq = np.asarray(X_train_seq, dtype=np.float32)
y_train_seq = np.asarray(y_train_seq, dtype=np.float32)


In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError

In [43]:
# Discriminator Model
def build_discriminator(input_shape):
    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True, input_shape=input_shape)),
        Dropout(0.2),
        Bidirectional(LSTM(64, return_sequences=False)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')  # Output probability (real or fake)
    ])
    return model

In [44]:
# Generator Model
def build_generator(input_shape):
    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True, input_shape=input_shape)),
        Dropout(0.2),
        Bidirectional(LSTM(64, return_sequences=False)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='linear')
    ])
    return model

In [59]:
# Training Parameters
learning_rate = 0.0001
batch_size = 1028
epochs = 30

In [46]:
# Input shape
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])

In [48]:
# Build models
generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)

# Optimizers
generator_optimizer = Adam(learning_rate)
discriminator_optimizer = Adam(learning_rate)

# Loss functions
generator_loss_fn = MeanAbsoluteError()
discriminator_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [53]:
print(len(X_train_seq))


403104


In [57]:
# Training Loop
def train_rgan(generator, discriminator, X_train_seq, y_train_seq, epochs, batch_size):
    total_samples = len(X_train_seq)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        
        # Shuffle data
        idx = np.random.permutation(total_samples)
        X_train_seq = X_train_seq[idx]
        y_train_seq = y_train_seq[idx]
        
        samples_processed = 0  # To track progress
        
        for i in range(0, total_samples, batch_size):
            # Get a batch of real data
            real_X = X_train_seq[i:i + batch_size]
            real_y = y_train_seq[i:i + batch_size]

            # Generate fake data
            noise = tf.random.normal([len(real_X), real_X.shape[1], real_X.shape[2]])
            fake_y = generator(noise, training=True)

            # Train discriminator
            with tf.GradientTape() as disc_tape:
                real_output = discriminator(real_X, training=True)
                fake_output = discriminator(noise, training=True)

                disc_loss_real = discriminator_loss_fn(tf.ones_like(real_output), real_output)
                disc_loss_fake = discriminator_loss_fn(tf.zeros_like(fake_output), fake_output)
                disc_loss = disc_loss_real + disc_loss_fake

            gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

            # Train generator
            with tf.GradientTape() as gen_tape:
                generated_y = generator(noise, training=True)
                gen_output = discriminator(noise, training=True)

                gen_loss = generator_loss_fn(real_y, generated_y)

            gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
            
            # Update samples processed and print progress if necessary
            samples_processed += len(real_X)
            if samples_processed % 2000 < batch_size:
                print(f"  Progress: {samples_processed}/{total_samples} samples processed.")

        # End of epoch summary
        print(f"Epoch {epoch + 1} Completed - Generator Loss: {gen_loss.numpy()}, Discriminator Loss: {disc_loss.numpy()}")


In [60]:
# Train the RGAN
train_rgan(generator, discriminator, X_train_seq, y_train_seq, epochs, batch_size)

Epoch 1/30
  Progress: 2056/403104 samples processed.
  Progress: 4112/403104 samples processed.
  Progress: 6168/403104 samples processed.
  Progress: 8224/403104 samples processed.
  Progress: 10280/403104 samples processed.
  Progress: 12336/403104 samples processed.
  Progress: 14392/403104 samples processed.
  Progress: 16448/403104 samples processed.
  Progress: 18504/403104 samples processed.
  Progress: 20560/403104 samples processed.
  Progress: 22616/403104 samples processed.
  Progress: 24672/403104 samples processed.
  Progress: 26728/403104 samples processed.
  Progress: 28784/403104 samples processed.
  Progress: 30840/403104 samples processed.
  Progress: 32896/403104 samples processed.
  Progress: 34952/403104 samples processed.
  Progress: 37008/403104 samples processed.
  Progress: 38036/403104 samples processed.
  Progress: 40092/403104 samples processed.
  Progress: 42148/403104 samples processed.
  Progress: 44204/403104 samples processed.
  Progress: 46260/403104 

KeyboardInterrupt: 